# Setup

In [44]:
import pickle
import pandas as pd

In [45]:
with open('homework/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [46]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [47]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

# Question 1 

What's the standard deviation of the predicted duration for this dataset?

In [48]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [49]:
q1_df_result = pd.DataFrame()
q1_df_result['predicted_duration'] = y_pred
print(q1_df_result['predicted_duration'].std())


5.281404481465351


# Question 2

In [50]:
year = 2022
month = 2
output_file = f'output/{year:04d}-{month:02d}.parquet'
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,ride_id
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,...,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333,2022/02_0
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,...,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000,2022/02_1
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,...,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000,2022/02_2
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,...,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333,2022/02_3
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,...,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667,2022/02_4


In [51]:
def save_results(df, y_pred, output_file):
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred

    df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
    )
    return df_result

In [52]:
df_result = save_results(df, y_pred, output_file)

In [53]:
df_result.head()

,ride_id,predicted_duration
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904


~60MB

# Question 3
jupyter nbconvert --to script week4_homework.ipynb

# Question 4
What's the first hash for the Scikit-Learn dependency?

sha256:065e9673e24e0dc5113e2dd2b4ca30c9d8aa2fa90f4c0597241c93b63130d233

# Question 5

```
df_result = ride_duration_prediction(
        run_date=datetime(year=year, month=month, day=1)
    )
print(df_result['predicted_duration'].mean())

In [43]:
!python week4_homework.py 3 2022

12.758556818790902


# Question 6

In order to get the april 2022 data from the docker container we can run it

docker run -it week4hw:v1 python week4_homework.py 4 2022

12.827242870079969